### 다중언어 개체명인식

- 제로샷 교차 언어 전이 (zero-shot cross lingual switching) 가능
  - 한 언어에서 파인 튜닝된 모델이 훈련없이 다른 모델에 적용 가능하다.
- 코드 스위칭 에 적합
  - 하나의 대화에서 둘이상의 언어나, 사투리등을 바꿈

- XTREME 데이터 셋 이용
    - PAN-X, wikiANN : 교차 언어 데이터 셋
    - LOC, PER, ORG
    - IOB2 포맷
        - B- 개체명의 시작
        - i- 동일 개체명의 연속
        - o- 어떤 개체에도 속하지 않음

In [5]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
"서브셋 갯수",len(xtreme_subsets)

('서브셋 갯수', 183)

In [30]:
for i,l in enumerate(xtreme_subsets):
    if "PAN" in l:
        print(l.split(".")[-1], end=': ')
        print(l, end=', ')
        if i % 5 == 0:
            print("")

af: PAN-X.af, ar: PAN-X.ar, 
bg: PAN-X.bg, bn: PAN-X.bn, de: PAN-X.de, el: PAN-X.el, en: PAN-X.en, 
es: PAN-X.es, et: PAN-X.et, eu: PAN-X.eu, fa: PAN-X.fa, fi: PAN-X.fi, 
fr: PAN-X.fr, he: PAN-X.he, hi: PAN-X.hi, hu: PAN-X.hu, id: PAN-X.id, 
it: PAN-X.it, ja: PAN-X.ja, jv: PAN-X.jv, ka: PAN-X.ka, kk: PAN-X.kk, 
ko: PAN-X.ko, ml: PAN-X.ml, mr: PAN-X.mr, ms: PAN-X.ms, my: PAN-X.my, 
nl: PAN-X.nl, pt: PAN-X.pt, ru: PAN-X.ru, sw: PAN-X.sw, ta: PAN-X.ta, 
te: PAN-X.te, th: PAN-X.th, tl: PAN-X.tl, tr: PAN-X.tr, ur: PAN-X.ur, 
vi: PAN-X.vi, yo: PAN-X.yo, zh: PAN-X.zh, 

### 언어 선택하기

- 선택 언어
    - PAN-X.en : 영어
    - PAN-X.ko : 한국어
    - PAN-X.ja : 일본어
    - PAN-X.es : 스페인어

In [98]:
from datasets import load_dataset
from collections import defaultdict
from datasets import DatasetDict

#일반적은 불균형 상태 만들기
langs = ["en", "ko", "ja", "es"]
fracs = [0.12, 0.6, 0.8, 0.1]

panx_ch = defaultdict(DatasetDict)

In [99]:
for lang, frac in zip(langs, fracs):
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in ds:
        panx_ch[lang][split]=(
            ds[split]
            .shuffle(seed=42)
            .select(range(int(frac*ds[split].num_rows))))


In [105]:
import pandas as pd

pd.DataFrame({lang : [panx_ch[lang]["train"].num_rows] for lang in langs}, index=["Samples"])

,en,ko,ja,es
Samples,2400,12000,16000,2000


In [153]:
element = panx_ch["ko"]["train"][0]

for k, v in element.items():
    print(k,v)

print("\nfeatures 중 ner-태그 확인")
ner_tags = panx_ch["ko"]["train"].features["ner_tags"].feature
ner_tags

tokens ['《트와일라잇》을', '같이', '찍은', '에디', '가테지', ',', '크리스틴', '스튜어트', ',', '로버트', '패틴슨과는', '매우', '친한사이라고', '한다', '.']
ner_tags [0, 0, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0]
langs ['ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko']

features 중 ner-태그 확인


ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [171]:
# NER-태그를 인덱스 에서 태그명 변경 및 추가

panx_ko = panx_ch["ko"].map(lambda x : {"ner_tag_names" :
                                        [ner_tags.int2str(idx)
                                        for idx in x["ner_tags"]]})
panx_ko

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 12000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 6000
    })
})

In [208]:
pd.DataFrame([panx_ko["train"][0]["tokens"],
              panx_ko["train"][0]["ner_tag_names"]],
              index=["tokens","ner_tag_name"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
tokens,《트와일라잇》을,같이,찍은,에디,가테지,",",크리스틴,스튜어트,",",로버트,패틴슨과는,매우,친한사이라고,한다,.
ner_tag_name,O,O,O,B-PER,I-PER,O,B-PER,I-PER,O,B-PER,I-PER,O,O,O,O


- 결과: 사람 이름에 태그가 붙었다.

### 태그 분포 확인하기

In [227]:
from collections import Counter

split2freqs = defaultdict(Counter)

for split , dataset in panx_ko.items():
    for row in dataset["ner_tag_names"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1

split2freqs

defaultdict(collections.Counter,
            {'train': Counter({'LOC': 7097, 'ORG': 5361, 'PER': 4870}),
             'validation': Counter({'LOC': 3579, 'ORG': 2755, 'PER': 2448}),
             'test': Counter({'LOC': 3503, 'ORG': 2584, 'PER': 2557})})